In [2]:
import boto3
import pandas as pd
from io import BytesIO

In [3]:
# Config cliente de S3
s3 = boto3.client('s3')

In [4]:
# Bbucket y prefijo de carpetas
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
prefix_ABTv2 = 'ABTv2/'
prefix_v1 = 'prueba_sk_forecast/'

In [13]:
# Obtengo lista de carpetas en el bucket
def list_folders(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    folders = [prefix['Prefix'].split('/')[-2] for prefix in response.get('CommonPrefixes', [])]
    return folders

In [6]:
# Llamo funcion para listar carpetas
folders_v2 = list_folders(bucket_name, prefix_ABTv2)

In [30]:
# Leo los xlsx y consolido

def read_files(bucket_name, prefix):
    dfs = []
    
    # Iterar sobre cada carpeta
    for folder_name in folders_v2:
        # Obtener la lista de objetos en la carpeta
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=f'{prefix}{folder_name}/')['Contents']
        
        # Buscar archivos xlsx en la carpeta
        excel_objects = [obj for obj in objects if obj['Key'].endswith('.xlsx')]
        
        if excel_objects:
            print(excel_objects)
            # Leer el primer archivo xlsx encontrado
            obj = s3.get_object(Bucket=bucket_name, Key=excel_objects[0]['Key'])
            excel_data = obj['Body'].read() # No se puede leer con pandas directo de S3
            
            df = pd.read_excel(BytesIO(excel_data))
            # Agrego columna con el nombre de la carpeta
            df['folder_name'] = folder_name
            # Agregar el df a la lista
            dfs.append(df)
        else:
            print(f"No se encontraron archivos xlsx en la carpeta {folder_name}.")
    
    if dfs:
        # Concateno todo
        consolidated_df = pd.concat(dfs, ignore_index=True)
        return consolidated_df
    else:
        print("No se encontraron archivos xlsx en ninguna de las carpetas especificadas.")
        return None

In [8]:
# Llamo funcion para leer xlsx y consolidar en un unico DF
df = read_files(bucket_name, prefix_ABTv2)

No se encontraron archivos xlsx en la carpeta FEDECREDITO (RYT)_EL SALVADOR.
No se encontraron archivos xlsx en la carpeta TOP 10 PAYERS (ENERO 2021-OCT 2023).
No se encontraron archivos xlsx en la carpeta jerarquico.


In [10]:
# Agrupar por payer y calcular el promedio de MAPE
df_ABTv2 = df.groupby('folder_name')['mape'].mean().reset_index()
df_ABTv2.rename(columns={'folder_name':'payer_country','mape': 'mapev2'}, inplace=True)

#### By Country

In [43]:
df['country'] = df['folder_name'].apply(lambda x: x.split('_')[-1])
df['date']=pd.to_datetime(df['date'])

In [44]:
df_country=df.groupby(['country','date']).agg({'valor_real': 'sum', 'valor_predicho': 'sum'}).reset_index()

In [45]:
###DESVIO ABSOLUTO POR DIA##
df_country['error_abs']= abs(df_country.valor_predicho - df_country.valor_real)

###MAPE DIA COUNTRY##
df_country['mape']= df_country.error_abs/df_country.valor_real

In [47]:
promedio_mape_por_country = df_country.groupby('country')['mape'].mean().reset_index()

In [48]:
promedio_mape_por_country

,country,mape
0,BRAZIL,0.190173
1,COLOMBIA,0.127298
2,DOMINICAN REPUBLIC,0.106475
3,EL SALVADOR,0.073179
4,GUATEMALA,0.079353
5,HONDURAS,0.067624
6,MEXICO,0.076095
7,PHILIPPINES,0.157279
